# Procedimento:

- Importare dati, isolare feature _'sii'_  (target feature)
- Rimuovere le feature presenti solo nel train e non test set
- Rimuovere feature con una percentuale di valori NaN >= 60%
- Splittare feature tra categoriche e numeriche
- Rimuovere record con una percentuale di valori NaN >= [ (considerando le 39 feature numeriche, definisco la percentuale come: numero feature Physical + numero feature FGC + numero feature BIA / 39 ) ] * 100
- Boxplot delle feature (suddivise per tipologia, BIA; Physical; FGC; etc.) per visualizzare gli outliers
- Rimuovo record che presenta grandi outliers (rischio di aumentare MSE del modello)
- [OPZIONALE] Provare a unificare feature simili o aggregabili (Physical_Height con Physical_Weight con Physical_BMI)
- [OPZIONALE] Capire se vale la pena unificare blocco FGC formato da FGC_attributo; FGC_attributo_zona
- Capire come riempire i record NaN (usare media? interpolazione? insiemi di medie?)
- Capito ciò allineare il riempimento con il tipo di dato presente in origine (se dato registrato è intero e riempio con x, se 0<=x < 0.5 floor(x); se 0.5 < x < 1 ceiling(x) )
- 1HE su feature categoriche
- Estraggo dataset

In [45]:
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np

load_dotenv()
TRAIN_SET = os.getenv("TRAIN_PATH")
TEST_SET  = os.getenv("TEST_PATH")

train = pd.read_csv(TRAIN_SET)
test = pd.read_csv(TEST_SET)


train.head()

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,...,4.0,2.0,4.0,55.0,NaN,NaN,NaN,Fall,3.0,2.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,...,0.0,0.0,0.0,0.0,Fall,46.0,64.0,Summer,0.0,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,...,2.0,1.0,1.0,28.0,Fall,38.0,54.0,Summer,2.0,0.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,3.0,4.0,1.0,44.0,Summer,31.0,45.0,Winter,0.0,1.0
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
test.head()

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,BIA-BIA_TBW,PAQ_A-Season,PAQ_A-PAQ_A_Total,PAQ_C-Season,PAQ_C-PAQ_C_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,...,32.6909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fall,3.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,...,27.0552,NaN,NaN,Fall,2.340,Fall,46.0,64.0,Summer,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,...,NaN,NaN,NaN,Summer,2.170,Fall,38.0,54.0,Summer,2.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,45.9966,NaN,NaN,Winter,2.451,Summer,31.0,45.0,Winter,0.0
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,...,NaN,Summer,1.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Import!**

In [47]:
import importlib
import utils as ut

importlib.reload(utils)

<module 'utils' from 'd:\\UniLectures\\.III\\DWM\\DWM\\utils.py'>

In [48]:
train = train.dropna(subset=['sii']) #removes all the records with NaN in the sii (target) column

y_train = train['sii']

y_train.shape

(2736,)

In [49]:
X_train = ut.intersect_features(train, test) #selects the features that are present in both train and test sets
X_train.shape

(2736, 58)

In [50]:
X_train.head()

,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,...,BIA-BIA_TBW,PAQ_A-Season,PAQ_A-PAQ_A_Total,PAQ_C-Season,PAQ_C-PAQ_C_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday
0,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,NaN,...,32.6909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fall,3.0
1,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,22.0,...,27.0552,NaN,NaN,Fall,2.340,Fall,46.0,64.0,Summer,0.0
2,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,NaN,...,NaN,NaN,NaN,Summer,2.170,Fall,38.0,54.0,Summer,2.0
3,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,NaN,...,45.9966,NaN,NaN,Winter,2.451,Summer,31.0,45.0,Winter,0.0
5,Spring,13,1,Winter,50.0,Summer,22.279952,59.5,112.2,NaN,...,63.1265,NaN,NaN,Spring,4.110,Summer,40.0,56.0,Spring,0.0


In [51]:
X_train, removed_features = ut.drop_columns(X_train, 0.4)
X_train.shape

(2736, 48)

In [52]:
X_train_numerical, X_train_categorical = ut.extract_numerical_cathegorical(X_train)

print("There are: ", len(X_train_numerical), "| Numerical Features -> ", X_train_numerical)
print("There are: ", len(X_train_categorical), " | Categorical Features -> ", X_train_categorical)

There are:  39 | Numerical Features ->  ['Basic_Demos-Age', 'Basic_Demos-Sex', 'CGAS-CGAS_Score', 'Physical-BMI', 'Physical-Height', 'Physical-Weight', 'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_PU', 'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR', 'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI', 'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM', 'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num', 'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM', 'BIA-BIA_TBW', 'PAQ_C-PAQ_C_Total', 'SDS-SDS_Total_Raw', 'SDS-SDS_Total_T', 'PreInt_EduHx-computerinternet_hoursday']
There are:  9  | Categorical Features ->  ['Basic_Demos-Enroll_Season', 'CGAS-Season', 'Physical-Season', 'Fitness_Endurance-Season', 'FGC-Season', 'BIA-Season', 'PAQ_C-Season', 'SDS-Season', 'PreInt_EduHx-Season']


In [53]:
#i want to isolate all the important features with missing values: Physical, FGC, BIA

feature_clusters = []

for feat in X_train_numerical:
    temp = feat.split('-')[0]
    if temp not in feature_clusters and temp in ['Physical', 'FGC', 'BIA']:
        feature_clusters.append(temp)
print(feature_clusters)

['Physical', 'FGC', 'BIA']


In [58]:
rows_clusters_information = ut.count_feature_clusters(X_train_numerical, feature_clusters)
print(rows_clusters_information)

{'Physical': 6, 'FGC': 10, 'BIA': 16}


In [59]:
total = 0
for key in rows_clusters_information:
    total += rows_clusters_information[key]
print(total)

32


In [62]:
rows_percentage = np.floor(total / len(X_train_numerical) * 100)
print(rows_percentage)

82.0


In [ ]:
#now i want to delete all the rows with more of 82% of missing values. Before that i want to reattach the target column to the dataset